In [45]:
import datetime
import sys
import pandas as pd
import numpy as np
import math
sys.path.append('../common/')
sys.path.append('../data/')
import warnings
pd.set_option("mode.chained_assignment", None)
# Suppress the specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, message="is_sparse is deprecated")
print(sys.version)

pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

3.10.6 (main, Oct  7 2022, 20:19:58) [GCC 11.2.0]


In [46]:
# read from csv
df_sequence_20230101 = pd.read_csv('../data/sequence_20230101.csv', sep='|')

/tmp/ipykernel_966/1416994791.py:2: DtypeWarning: Columns (80) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sequence_20230101 = pd.read_csv('../data/sequence_20230101.csv', sep='|')


In [47]:
df_sequence_20230101

,OCCURRED_DATE,SEQUENCE_ID,SEQUENCE_NAME,STEP_ID,TEMPLATE_ID,SEQUENCE_ENROLLMENT_ID,SEQUENCE_STEP_EXCUTION_ID,CONTACT_ID,CONTACT_FIRST_NAME,CONTACT_LAST_NAME,CONTACT_COMPANY_ID,CONTACT_COMPANY_NAME,SENT_COUNT,REPLY_COUNT,OPEN_COUNT,MEETING_BOOKED_COUNT,CONTACT_UPDATE_DATE,CONTACT_CITY,CONTACT_COMPANY,CONTACT_COUNTRY,CONTACT_CREATE_DATE,CONTACT_CREATE_TIMESTAMP,CONTACT_EAMIL,CONTACT_FIRST_DEAL_CREATE_DATE,CONTACT_ADDITIONAL_EMAIL,CONTACT_EMAIL_DOMAIN,CONTACT_IP_TIMEZONE,CONTACT_IS_UNWORKED,CONTACT_LATEST_SEQUENCE_ENDED_DATE,CONTACT_LATEST_SEQUENCE_ENDED_TIMESTAMP,CONTACT_LATEST_SEQUENCE_ENROLLED_DATE,CONTACT_LATEST_SEQUENCE_ENTROLLED_TIMESTAMP,CONTACT_LEAD_STATUS,CONTACT_SEQUENCES_ENROLLED_COUNT,CONTACT_SEQUENCES_IS_ENROLLED,CONTACT_TIME_BETWEEN_CONTACT_CREATION_AND_DEAL_CREATION,CONTACT_HUBSPOT_OWNER_ID,CONTACT_HUBSPOT_TEAM_ID,CONTACT_JOB_TITLE,CONTACT_LAST_MODIFIED_DATE,CONTACT_LAST_MODIFED_TIMESTAMP,CONTACT_LEAD_SOURCE,CONTACT_LINKEDIN,CONTACT_MOBILE_PHONE,CONTACT_NUM_ASSOCIATED_DEALS,CONTACT_NUM_CONTACTED_NOTES,CONTACT_PHONE,CONTACT_STATE,CONTACT_UNITS,CONTACT_WEBSITE,CONTACT_ZIPCODE,COMPANY_OBJECT_TYPE_ID,COMPANY_UPDATE_DATE,COMPANY_ADDRESS,COMPANY_ADDRESS2,COMPANY_ANNUAL_REVENUE,COMPANY_CITY,COMPANY_COUNTRY,COMPANY_CREATE_DATE,COMPANY_CREATE_TIMESTAMP,COMPANY_DOMAIN,COMPANY_FACEBOOK_PAGE,COMPANY_FIRST_CONTACT_DATE,COMPANY_FOUNDED_YEAR,COMPANY_LEAD_STATUS,COMPANY_LINKEDIN_PAGE,COMPANY_NAME,COMPANY_NUM_ASSOCIATED_CONTACTS,COMPANY_NUM_ASSOCIATED_DEALS,COMPANY_NUM_PROPERTIES,COMPANY_PHONE,COMPANY_PM_SOFTWARE,COMPANY_PROTFOLIO_SUBTYPE,COMPANY_PROTFOLIO_TYPE,COMPANY_RENT_PORTAL,COMPANY_SEGMENT,COMPANY_STATE,COMPANY_TIMEZONE,COMPANY_UNITS,COMPANY_WEBSITE,COMPANY_ZIPCODE
0,20230410,76684087,Portal Sequence,0,37999983,6732583864,76684087-1568041278-6732583864,35548,Chris,Honeycutt,4280730952,Constellation Property Management,1.0,NaN,NaN,NaN,2023-10-10 19:38:25.344,San Diego,Constellation Property Management,NaN,2020-08-06 21:32:29.807,1596749549807,ch@conmgmt.com,NaN,NaN,conmgmt.com,NaN,False,2023-09-29 18:35:24.236,1.696013e+12,2023-09-18 19:47:52.913,1695066472913,Reached Out,5,False,NaN,473513267,34335131,President/Broker,2023-10-10 19:38:25.344,1696966705344,NaN,NaN,+18583364700,NaN,21,+1 (619) 955-7776,CA,247.0,http://conmgmt.com/Constellation_Realty_Manage...,92101,0-2,2023-10-07 00:01:23.259,1901 1st Ave,NaN,1000000.0,San Diego,United States,2020-08-05 17:04:12.334,1596647052334,conmgmt.com,NaN,2020-08-06 21:32:25.243,2013.0,Active,NaN,Constellation Property Management,8,NaN,3.0,+1 (619) 955-7776,NaN,NaN,Multi Family,NaN,Deep SMB,CA,America/Los_Angeles,247,http://conmgmt.com/Constellation_Realty_Manage...,92101
1,20230922,90434194,Micro Segment 1 - Deep SMB - < 250,2,48653112,8045771370,90434194-1858371402-8045771370,35548,Chris,Honeycutt,4280730952,Constellation Property Management,1.0,NaN,NaN,NaN,2023-10-10 19:38:25.344,San Diego,Constellation Property Management,NaN,2020-08-06 21:32:29.807,1596749549807,ch@conmgmt.com,NaN,NaN,conmgmt.com,NaN,False,2023-09-29 18:35:24.236,1.696013e+12,2023-09-18 19:47:52.913,1695066472913,Reached Out,5,False,NaN,473513267,34335131,President/Broker,2023-10-10 19:38:25.344,1696966705344,NaN,NaN,+18583364700,NaN,21,+1 (619) 955-7776,CA,247.0,http://conmgmt.com/Constellation_Realty_Manage...,92101,0-2,2023-10-07 00:01:23.259,1901 1st Ave,NaN,1000000.0,San Diego,United States,2020-08-05 17:04:12.334,1596647052334,conmgmt.com,NaN,2020-08-06 21:32:25.243,2013.0,Active,NaN,Constellation Property Management,8,NaN,3.0,+1 (619) 955-7776,NaN,NaN,Multi Family,NaN,Deep SMB,CA,America/Los_Angeles,247,http://conmgmt.com/Constellation_Realty_Manage...,92101
2,20230424,76684087,Portal Sequence,2,38000003,6732583936,76684087-1568041278-6732583936,35548,Chris,Honeycutt,4280730952,Constellation Property Management,1.0,NaN,NaN,NaN,2023-10-10 19:38:25.344,San Diego,Constellation Property Management,NaN,2020-08-06 21:32:29.807,1596749549807,ch@conmgmt.com,NaN,NaN,conmgmt.com,NaN,False,2023-09-29 18:35:24.236,1.6

In [48]:
#fill None value of performance with 0
subset = ['SENT_COUNT', 'REPLY_COUNT', 'OPEN_COUNT', 'MEETING_BOOKED_COUNT']
df_sequence_20230101.loc[:, subset] = df_sequence_20230101.loc[:, subset].fillna(0).astype(int)

In [165]:
seq_perf_df = (df_sequence_20230101.groupby(by=['SEQUENCE_ID'])
                          .agg(send_count=('SENT_COUNT', 'sum'),
                               reply_count=('REPLY_COUNT', 'sum'),
                               meeting_booked_count=('MEETING_BOOKED_COUNT', 'sum'),
                               open_count=('OPEN_COUNT', 'sum'),
                               enrolled_contact_count=('CONTACT_ID', 'nunique'),
                               sequence_name=('SEQUENCE_NAME', 'first')
                               )).reset_index()

# filter out the sequence barely used in 2023
seq_perf_df = seq_perf_df[seq_perf_df['enrolled_contact_count'] > 50]
seq_perf_df['reply_rate'] = seq_perf_df['reply_count'] / seq_perf_df['send_count']
seq_perf_df['meeting_booked_rate'] = seq_perf_df['meeting_booked_count'] / seq_perf_df['send_count']
seq_perf_df.sort_values(by=['reply_rate'], ascending=False, inplace=True)
seq_perf_df


# !! Need to exclude some invalid sequence
status_special_words = ['Engagement', 'resident', 'Re-Intro']
status_special_df = seq_perf_df[seq_perf_df['sequence_name'].apply(lambda x: True if any(ssw.lower() in x.lower() for ssw in status_special_words) else False)][['SEQUENCE_ID', 'sequence_name']]
status_special_df

event_special_words = ['Holiday Delinquency', 'holiday', 'NARPM', 'BP', 'Rent Reporting', 'TAA', 'NAA', 'AIM']
event_special_df = seq_perf_df[seq_perf_df['sequence_name'].apply(lambda x: True if any(esw.lower() in x.lower() for esw in event_special_words) else False)][['SEQUENCE_ID', 'sequence_name']]
event_special_df

extra_exclude = [

{'SEQUENCE_ID': 75585648, 'sequence_name': "White Space Mar - Cory/Louis"}, #already have user in pmc, reachout other properties 
{'SEQUENCE_ID': 90434194, 'sequence_name': "Micro Segment 1 - Deep SMB - < 250"}, #with NARPM related template  
{'SEQUENCE_ID': 77550862, 'sequence_name': "Katie/Mina Feb 2023"}, #for previous touched pmc  
{'SEQUENCE_ID': 91000942, 'sequence_name': "Phase 2 Micro Segment 2 - Deep SMB - 250-500"}, # with NARPM related template 
{'SEQUENCE_ID': 90430277, 'sequence_name': "Micro Segment 3 - Deep SMB - 501-999"}, # with NARPM related template  
{'SEQUENCE_ID': 91009671, 'sequence_name': "Phase 2 Micro Segment 3 - Deep SMB - 501-999"}, # with NARPM related template 
]
extra_df = pd.DataFrame(extra_exclude)
extra_df

exclude_df = pd.concat([status_special_df, event_special_df, extra_df]).reset_index()
display(exclude_df)

# filter out the excluded sequence 

seq_perf_df = seq_perf_df[~seq_perf_df['SEQUENCE_ID'].isin(exclude_df['SEQUENCE_ID'])]

# filter out low performance sequence 
seq_perf_df = seq_perf_df[seq_perf_df['reply_rate']]

# combine similar sequence 
seq_perf_df

,index,SEQUENCE_ID,sequence_name
0,84,80362343,April '23 Engagement Outreach - Corporate (Louis)
1,39,76742756,Feb. '23 Engagement Outreach - Corporate Fiona...
2,70,78333524,Louis - May '23 Engagement Outreach
3,222,91280508,Cold outreach 1: For a property with 1 residen...
4,204,90611753,Cold outreach 1: For a property with 1 residen...
5,38,76737910,Feb. '23 Engagement Outreach - Property Level ...
6,212,91009600,Cold outreach 1: For a property with 1 residen...
7,181,89930083,Re-Intro
8,210,91000972,Cold outreach 1: For a property with 1 residen...
9,223,91283570,Cold outreach 1: For a property with 1 residen...


KeyError: "None of [Float64Index([  0.07086614173228346,  0.046819787985865724,\n               0.036978341257263604,   0.03134796238244514,\n                0.02824858757062147,  0.025688073394495414,\n                0.02564102564102564,   0.02356020942408377,\n                0.02249134948096886,   0.02075765438505449,\n                0.01932367149758454,  0.018613680484184082,\n               0.017857142857142856,    0.0170316301703163,\n                0.01560758082497213,  0.015544041450777202,\n                0.01532567049808429,   0.01488833746898263,\n               0.014522821576763486,  0.014332965821389196,\n               0.014204545454545454,  0.013793103448275862,\n               0.013559322033898305,  0.013368983957219251,\n               0.013215859030837005,  0.012944983818770227,\n               0.012863070539419087,    0.0121580547112462,\n               0.012145748987854251,  0.011515151515151515,\n               0.011049723756906077,  0.010948905109489052,\n    

In [166]:
# combine similar sequence 

def seq_name_cat(seq_name):
    if '2nd Multi Touch'.lower() in seq_name.lower():
        return '2nd Multi Touch'
    if any( word.lower() in seq_name.lower() for word in ['Fiona/Jordan Feb seq', 'Portal', 'JE Jan 23 with calls', 
                                          'Jordan Feb Sequence', 'Garrett/Fiona (ALN)', 'Fiona/Ariel April (clone)', 
                                          'JE Jan 23 with calls Clone - no info', 'Ally/Mina April Seq', 'JE Jan 23', 'Activate Flex', 'Brandon/Louis July - portal activation Clone'
                                         ]):
        return 'Portal Sequence'
    if any( word.lower() in seq_name.lower() for word in ['Benefits']):
        return 'Benefits'
    if any( word.lower() in seq_name.lower() for word in ['Jan 2023', 'January 2023']):
        return 'New Year Start'
    if any( word.lower() in seq_name.lower() for word in ['Regional']):
        return 'Regionals'
    if any( word.lower() in seq_name.lower() for word in ['chatgpt']):
        return 'ChatGPT'
    return seq_name
    
seq_perf_df['sequence_name_cat'] = seq_perf_df['sequence_name'].apply(lambda x: seq_name_cat(x))
seq_perf_df

,SEQUENCE_ID,send_count,reply_count,meeting_booked_count,open_count,enrolled_contact_count,sequence_name,reply_rate,meeting_booked_rate,sequence_name_cat
14,73453658,127,9,0,33,125,Nov. Seq.,0.070866,0.000000,Nov. Seq.
5,64225429,1132,53,1,475,409,2nd Multi Touch,0.046820,0.000883,2nd Multi Touch
2,37380911,1893,70,0,911,533,Spencer Multi Touch Master Sequence,0.036978,0.000000,Spencer Multi Touch Master Sequence
79,79983031,319,10,0,118,85,"March/April 23- calls, Allison",0.031348,0.000000,"March/April 23- calls, Allison"
192,90434902,177,5,0,65,71,Micro Segment 2 - Deep SMB - 250-500,0.028249,0.000000,Micro Segment 2 - Deep SMB - 250-500
56,77404520,1090,28,0,221,408,Cory/Louis Feb 2023,0.025688,0.000000,Cory/Louis Feb 2023
104,82137201,780,20,0,183,182,Portal Sequence JM,0.025641,0.000000,Portal Sequence
153,87748694,382,9,0,115,136,Ally/Mina June/July,0.023560,0.000000,Ally/Mina June/July
47,77065378,578,13,0,236,214,JE Jan 23 with calls,0.022491,0.000000,Portal Sequence
36,76684087,9635,200,0,4138,2344,Portal Sequence,0.020758,0.000000,Portal Sequence


In [135]:
## userful arguments 
# https://app.hubspot.com/templates/7415199/edit/40575824 
# Our partner, Asset Living saw a 4% reduction in delinquency after implementing Flex. -- successful story

# any holiday related staff?

# previous emailed but no response 

# portal specific?  https://app.hubspot.com/templates/7415199/edit/42052102 FL Portal Sequence from portal perspective

# REVIEWs seems not useful?# REVIEWs seems not useful?